# Lab3.4 Sentiment Classification using transformer models

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook explains how you can use a transformer model that is fine-tuned for sentiment analysis. Fine-tuned transformer models are published regularly on the huggingface platform: https://huggingface.co/models

These models are very big (Gigabytes) and require a computer with sufficient memory to load. Furthermore, loading these models takes some time as well. It is also possible to copy such a model to your disk and to load the local copy. Still a substantial memory is needed to load it.

This notebook requires installing some deep learning packages: transformers, pytorch and simpletransformers. If you are not experienced with installing these packages, make sure you first define a virtual environment for python, activate this environment and install the packages in this enviroment.

Please consult the Python documentation for installing such an enviroment:

https://docs.python.org/3/library/venv.html

After activating your enviroment you can install pytorch, transformers and simpletransformers from the command line. If you start this notebook within the same virtual environment you can also execute the next installation commands from your notebook. Once installed, you can comment out the next cell.

In [ ]:
#!conda install pytorch cpuonly -c pytorch
#!pip install transformers
#!pip install simpletransformers

Huggingface transfomers provides an option to create a **pipeline** to perform a NLP task with a pretrained model: 

"The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering."

More information can be found here: https://huggingface.co/transformers/v3.0.2/main_classes/pipelines.html

We will use the pipeline module to load a fine-tuned model to perform senteiment analysis

In [ ]:
from transformers import pipeline

We load a transformer model 'distilbert-base-uncased-finetuned-sst-2-english' that is fine-tuned for binary classification from the Hugging face repository:

https://huggingface.co/models

We need to load the model for the sequence classifcation and the tokenizer to convert the sentences into tokens according to the vocabulary of the model.

Loading the model takes some time.

In [ ]:
sentimentenglish = pipeline("sentiment-analysis", 
                            model="distilbert-base-uncased-finetuned-sst-2-english", 
                            tokenizer="distilbert-base-uncased-finetuned-sst-2-english")

We now created an instantiation of a pipeline that can tokenize any sentence, obtain a sententence embedding from the transformer language model and perform the **sentiment-analysis** task. Let's try it out on an example sentence.

In [ ]:
sentence_pos_en = "Nice hotel and the service is great"

In [ ]:
sentimentenglish(sentence_pos_en)

In [ ]:
sentence_neg_en = "The rooms are dirty and the wifi does not work"

In [ ]:
sentimentenglish(sentence_neg_en)

This is easy and seems to work very well. 

## Using a Dutch fine-tuned transformer model

We can use a fine-tuned Dutch model for Dutch sentiment analysis by creating another pipeline. Again loading this model takes some time. Also note that after loading, both moodels are loaded in memory. So if you have issues loading, you may want to start over and try again just with the Dutch pipeline.

In [ ]:
sentimentdutch = pipeline("sentiment-analysis", 
                          model="wietsedv/bert-base-dutch-cased-finetuned-sentiment", 
                          tokenizer="wietsedv/bert-base-dutch-cased-finetuned-sentiment")

We test it on two similar Dutch sentences:

In [ ]:
sentence_pos_nl="Mooi hotel en de service is geweldig"
sentence_neg_nl="De kamers zijn smerig en de wifi doet het niet"

In [ ]:
sentimentdutch(sentence_pos_nl)

In [ ]:
sentimentdutch(sentence_neg_nl)

This seems to work fine too although the score for negative in the second example is much lower.

## Inspecting sentence representations using Simpletransformers

The Simpletransformers package is built on top of the transformer package. It simplifies the use of transformers even more and provides excellent documentation: https://simpletransformers.ai

The site explains also how you can fine-tune models yourself or even how to build models from scratch, assuming you have the computing power and the data.

Here we are going to use it to inspect the sentence representations a bit more. Unfortunately, we need to load the English model again as an instantiation of a RepresentationModel. So if you have memory issues, please stop the kernel and start again from here.

Loading the model may gave a lot of warnings. You can ignore these. If you do not have a graphical card (GPU) and or cuda installed to use the GPU you need to set use_cuda to False, as shown below.

In [ ]:
from simpletransformers.language_representation import RepresentationModel
        
#sentences = ["Example sentence 1", "Example sentence 2"]
model = RepresentationModel(
        model_type="bert",
        model_name="distilbert-base-uncased-finetuned-sst-2-english",
        use_cuda=False ## If you cannot use a GPU set this to false
    )

The Representationmodel allows you to obtain a sentence encoding. We do that next for the positive English example which consists of 7 words:

In [ ]:
sentence_pos_en

According to the simpletransformers API the input must be a list even when it is a single sentence. If you pass a string as input, it will turn it into a list of charcaters, each character as a separate sentence.

In [ ]:
word_vectors = model.encode_sentences([sentence_pos_en], combine_strategy=None)

The result is a numpy array with the shape (1, 9, 768) 

In [ ]:
print(type(word_vectors))
print(word_vectors.shape)

The first number indicates the number of sentences, which is **1** in our case. The next digit **9** indicates the number of tokens and the final digit is the number of dimension for each token according to the transformer model, which **768** in case of BERT models.

We can ask for the full embedding representation for the first token:

In [ ]:
print('Nr of dimensions for the mebdding of the first token:', len(word_vectors[0][0]))
print(word_vectors[0][0])

**WAIT** Our sentence has 7 words so why do we get 9 tokens here?

We can  use the tokenizer of the model to get the token representation of the transformer and check it out.

In [ ]:
tokenized_sentence = model.tokenizer(sentence_pos_en)
tokenized_sentence

Although our sentence has 7 words, we get 9 identifiers. We can use the **decode** function to convert them back to words:

In [ ]:
model.tokenizer.decode(101)

The first token is the special token **CLS** which is an abstract sentence representation. Let's check another one:

In [ ]:
model.tokenizer.decode(3309)

Allright, this a word from our sentence. Let's decode them all:

In [ ]:
tokenid_list = tokenized_sentence['input_ids']
for token_id in tokenid_list:
    print(token_id, model.tokenizer.decode(token_id))

The transformer model added the special tokens **CLS** and **SEP** but also represented our "Nice" with the **UNK** token. Any idea why? Check the name of the model we used.....

We used the uncased model, which means that for training all inoput was downcased.

# End of this notebook